In [15]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Amazon, Planetoid, PPI
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv
import numpy as np

def get_data(dataset):
    if dataset in ["Photo", "Computers"]:
        path = osp.join('data', dataset)
        dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())
        dataset.shuffle()
        config = {"l2": 0.0005, "heads": 1}
        return dataset, config
    elif dataset in ["Cora", "Citeseer"]:
        path = osp.join('data', dataset)
        dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
        config = {"l2": 0.0005, "heads": 1}
        return dataset, config
    elif dataset == "Pubmed":
        path = osp.join('data', dataset)
        dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
        config = {"l2": 0.001, "heads": 8}
        return dataset, config
    elif dataset == "PPI":
        for split in ["train", "val", "test"]:    
            path = osp.join('data', dataset)
            dataset = PPI(path, split, transform=T.NormalizeFeatures())
        return dataset
    else:
        print("Unsupported Dataset")

In [13]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, config):
        super(Net, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(8 * 8, out_channels, heads=config["heads"], concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)
    
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs

In [20]:
values = []
for dataset in ["Citeseer"] * 100:
    dataset, config = get_data(dataset)
    TRAIN_END = 20 * dataset.num_classes
    VAL_END = TRAIN_END + 500
    TEST_END = VAL_END + 1000
    data = dataset[0]
    data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
    data.train_mask[:TRAIN_END] = True
    data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
    data.val_mask[TRAIN_END:VAL_END] = True
    data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
    data.test_mask[VAL_END:TEST_END] = True
    
    #data.edge_index = torch.empty((2,0), dtype=torch.long)  # no edges
    #data.x = torch.empty((data.num_nodes,data.num_features)) # no features
    
    FIXED_FEAT_VALUE = 1/745
    #data.x = torch.full((data.num_nodes,data.num_features), FIXED_FEAT_VALUE, dtype=torch.float) # fixed features

    #orch.manual_seed(0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)

    model = Net(dataset.num_features, dataset.num_classes, config).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=config["l2"])

    print()
    print(dataset)
    avg = np.empty([1,])
    max_val = 0
    early_stop_cnt = 0
    for epoch in range(1, 1001):
        train(data)
        train_acc, val_acc, test_acc = test(data)

        if val_acc > max_val:
            early_stop_cnt = 0
            max_val = val_acc
        else:
            early_stop_cnt += 1
            if (early_stop_cnt == 100):
                print(f'Early Stopping at epoch {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
                values = values+[test_acc]
                break

        avg = np.append(avg, test_acc)
        last_hun = np.average(avg[len(avg)-100:])
    

        #print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
        #      f'Test: {test_acc:.4f}, Avg100: {last_hun:.4f}')


Citeseer()
Early Stopping at epoch 150, Val: 0.7100, Test: 0.6730

Citeseer()
Early Stopping at epoch 130, Val: 0.7040, Test: 0.6830

Citeseer()
Early Stopping at epoch 126, Val: 0.7120, Test: 0.6880

Citeseer()
Early Stopping at epoch 132, Val: 0.7060, Test: 0.6700

Citeseer()
Early Stopping at epoch 235, Val: 0.6960, Test: 0.6930

Citeseer()
Early Stopping at epoch 239, Val: 0.7060, Test: 0.6940

Citeseer()
Early Stopping at epoch 140, Val: 0.7200, Test: 0.6890

Citeseer()
Early Stopping at epoch 130, Val: 0.7040, Test: 0.6800

Citeseer()
Early Stopping at epoch 131, Val: 0.6820, Test: 0.6580

Citeseer()
Early Stopping at epoch 127, Val: 0.7080, Test: 0.6860

Citeseer()
Early Stopping at epoch 142, Val: 0.6980, Test: 0.6880

Citeseer()
Early Stopping at epoch 143, Val: 0.6940, Test: 0.6720

Citeseer()
Early Stopping at epoch 155, Val: 0.7060, Test: 0.6740

Citeseer()
Early Stopping at epoch 117, Val: 0.7000, Test: 0.6870

Citeseer()
Early Stopping at epoch 126, Val: 0.7260, Test: 0.

In [21]:
print(np.mean(values))
print(np.std(values))

0.681650003194809
0.010799421390967531
